<a href="https://colab.research.google.com/github/Zan-Huang/PaperMeasure/blob/master/PaperMeasure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

import numpy as np
import time

import nltk
import scipy

from google.colab import drive
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from keras.models import load_model
from keras.models import model_from_json

from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
drive.mount('/content/gdrive')
!ls

!ls '/content/gdrive/My Drive/TMP'

!cd '/content/gdrive/My Drive/TMPMEASURE'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
gdrive	model.h5  model.json  modelscheck.hdf5	sample_data
 36.2.pdf   36.3.pdf  ' 36.4.pdf'   36.5.pdf   36.6.pdf   36.7.pdf   36.8.pdf


In [0]:
!rm 'model.json'
!rm 'model.h5'

In [0]:
!apt-get install build-essential libpoppler-cpp-dev pkg-config python-dev

In [0]:
text = "The problem of an appropriate bibliography has proven to be of the hardest. To the best of the author's belief, there exists, aside from that here given, no bibliography aiming at a systematic classification of the sources and discussions of the mythology of the Latin-American Indians, as a whole. There[Pg viii] are, indeed, a considerable number of special bibliographies, regional in character, for which every student must be grateful; and it is hoped that not many of the more important of these have failed of inclusion in the bibliographical division devoted to; but for the whole field, the appended bibliography is pioneer work, and subject to the weaknesses of all such attempts. The principles of inclusion are: (1) All works upon which the text of the volume directly rests. These will be found cited in the Notes, where are also a few references to works cited for points of an adventitious character, and therefore not included in the general bibliography. (2) A more liberal inclusion of English and Spanish than of works in other languages, the one for accessibility, the other for source importance. (3) An effort to select only such works as have material directly pertinent to the mythology, not such as deal with the general culture, of the peoples under consideration,—a line most difficult to draw. In respect to bibliography, it should be further stated that it is the intent to enter the names of Spanish authors in the forms approved by the rules of the Real Academia, while it has not seemed important to follow other than the English custom in either text or notes. It is certainly the author's hope that the labour devoted to the assembling of the bibliography will prove helpful to students generally, and it is his belief that those wishing an introduction to the more important sources for the various regions will find of immediate help the select bibliographies given in the Notes, for each region and chapter."

In [0]:
grammar_token_refined = []

def nltk_word(string_text):
  word_token = nltk.word_tokenize(string_text)
  grammar_token = nltk.pos_tag(word_token)
  for i in range(len(grammar_token)):
    grammar_token_refined.append(grammar_token[i][1])
  return word_token, grammar_token_refined

In [0]:
answer, grammar = nltk_word(text)
word_dictionary = sorted(set(answer))
grammar_dictionary = sorted(set(grammar))

wordcount_vector = dict.fromkeys(word_dictionary, 0)
grammarcount_vector = dict.fromkeys(grammar_dictionary, 0)

def word_count(word_size, grammar_size):
  for i in range(len(word_size)):
    for j in range(len(word_dictionary)):
      if (word_size[i] == word_dictionary[j]):
        wordcount_vector[word_dictionary[j]] = wordcount_vector[word_dictionary[j]] + 1
  for i in range(len(grammar_size)):
    for j in range(len(grammar_dictionary)):
      if (grammar[i] == grammar_dictionary[j]):
        grammarcount_vector[grammar_dictionary[j]] = grammarcount_vector[grammar_dictionary[j]] + 1 
  
  return wordcount_vector, grammarcount_vector
  
final_input_word, final_input_grammar = word_count(answer, grammar)
print(final_input_word)
print(final_input_grammar)

{'all': 1, 'assembling': 1, 'help': 1, 'enter': 1, 'indeed': 1, 'Indians': 1, 'wishing': 1, 'sources': 2, 'attempts': 1, 'grateful': 1, 'follow': 1, 'find': 1, 'seemed': 1, 'devoted': 2, 'regions': 1, 'certainly': 1, 'bibliography': 6, 'field': 1, '(': 3, 'appended': 1, 'character': 2, ',': 21, 'should': 1, 'forms': 1, 'failed': 1, 'only': 1, 'other': 3, 'text': 2, 'has': 2, 'hope': 1, 'division': 1, 'his': 1, 'to': 13, 'than': 2, 'rules': 1, 'Notes': 2, 'material': 1, 'every': 1, 'not': 4, 'aiming': 1, 'pertinent': 1, 'helpful': 1, 'notes': 1, 'with': 1, 'either': 1, 'each': 1, 'found': 1, 'works': 4, 'where': 1, 'exists': 1, 'principles': 1, 'culture': 1, 'Pg': 1, 'discussions': 1, 'further': 1, 'best': 1, 'subject': 1, 'for': 7, 'while': 1, 'pioneer': 1, '3': 1, 'various': 1, 'it': 5, ';': 2, 'be': 4, 'weaknesses': 1, 'belief': 2, 'importance': 1, 'here': 1, 'English': 2, 'consideration': 1, '[': 1, 'by': 1, 'chapter': 1, 'rests': 1, 'many': 1, 'region': 1, 'whole': 2, 'or': 1, 'cla

In [0]:
train_list = []
grammar_list = []
for key, value in wordcount_vector.iteritems():
  train_list.append(value)
  
for key, value in grammarcount_vector.iteritems():
  grammar_list.append(value)

  
X_raw = np.array(train_list)
X_normalized = np.log(X_raw)

grammar_raw = np.array(grammar_list)
grammar_normalized = np.log(grammar_raw)

print(X_normalized)
print(grammar_normalized)

[0.         0.         0.         0.         0.         0.
 0.         0.69314718 0.         0.         0.         0.
 0.         0.69314718 0.         0.         1.79175947 0.
 1.09861229 0.         0.69314718 3.04452244 0.         0.
 0.         0.         1.09861229 0.69314718 0.69314718 0.
 0.         0.         2.56494936 0.69314718 0.         0.69314718
 0.         0.         1.38629436 0.         0.         0.
 0.         0.         0.         0.         0.         1.38629436
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         1.94591015 0.         0.
 0.         0.         1.60943791 0.69314718 1.38629436 0.
 0.69314718 0.         0.         0.69314718 0.         0.
 0.         0.         0.         0.         0.         0.69314718
 0.         0.         1.09861229 0.         0.         0.
 0.         0.         0.         0.69314718 0.         2.19722458
 0.         0.69314718 0.         0.69314718 0.         0.
 0.         0.         0

Here, we parsed whatever text file was fed by the PDF and then created a bag of words, unnormalized however. The bag of words will be normalized later after a numpy array conversion by the size of the corpus.

**THIS IS ALL THE DATA WE NEED, ZAN JUST PUT THE FILE I SENT YOU IN THE COLAB.**

In [0]:
dataIn = np.load("/content/gdrive/My Drive/TMPMEASURE/preppedData.npy")
dataOut = np.load("/content/gdrive/My Drive/TMPMEASURE/preppedDataKey.npy")

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataIn, dataOut, test_size=0.10)

size_of_input = dataIn.shape[1]
print x_train.size
print y_train

18244980
[[0 1]
 [0 1]
 [1 0]
 ...
 [1 0]
 [0 1]
 [1 0]]


In [0]:
!rm 'modelscheck.hdf5'

In [0]:
model = Sequential()

model.add(Dense(units = 4000,  kernel_initializer='glorot_normal', bias_initializer='zeros', activation='relu', input_dim=size_of_input))
model.add(Dense(units = 1000, kernel_initializer='glorot_normal', bias_initializer='zeros', activation='relu'))
model.add(Dense(units = 50, kernel_initializer='glorot_normal', bias_initializer='zeros', activation='relu'))
model.add(Dense(2, activation='softmax'))

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

model_json = model.to_json()

checkpoint = [ModelCheckpoint(filepath='modelscheck.hdf5')]


with open("model.json", "w") as json_file:
    json_file.write(model_json)

# make sure you save the model zan!

In [85]:
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

scores = model.evaluate(x_test, y_test, verbose=1)

model.fit(x_train, y_train,
          validation_data = (x_test, y_test),
          epochs=8,
          batch_size=1,
          callbacks = checkpoint,
          verbose=1)

model.save('PaperMeasure/model.h5')

78/78 [==============================] - 0s 4ms/step
Train on 702 samples, validate on 78 samples
Epoch 1/8
702/702 [==============================] - 92s 131ms/step - loss: 0.4486 - acc: 0.7991 - val_loss: 0.2984 - val_acc: 0.8846
Epoch 2/8
702/702 [==============================] - 89s 127ms/step - loss: 0.0559 - acc: 0.9886 - val_loss: 0.3738 - val_acc: 0.9231
Epoch 3/8
702/702 [==============================] - 90s 128ms/step - loss: 0.0738 - acc: 0.9872 - val_loss: 1.2913 - val_acc: 0.8462
Epoch 4/8
702/702 [==============================] - 89s 127ms/step - loss: 0.0589 - acc: 0.9886 - val_loss: 0.9852 - val_acc: 0.8974
Epoch 5/8
702/702 [==============================] - 89s 127ms/step - loss: 0.0716 - acc: 0.9886 - val_loss: 1.0931 - val_acc: 0.8846
Epoch 6/8
702/702 [==============================] - 89s 127ms/step - loss: 0.0716 - acc: 0.9829 - val_loss: 0.7333 - val_acc: 0.8846
Epoch 7/8
702/702 [==============================] - 89s 127ms/step - loss: 2.7409e-04 - acc: 1.00

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
print(scores[1])

78/78 [==============================] - 0s 833us/step
0.8461538430971977


In [88]:

%ls

LICENSE  model.h5  PaperMeasure  README.md


In [0]:
!git add model.h5

In [92]:
!git commit -m "FINALLY"

[master d137a01] FINALLY
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 model.h5


In [0]:


!sshpass -p "zanhuang" scp 'model.md5' alex@dp.ton.io:/